Based off of work in vocab.ipynb, this program extracts tobacco and drug related bigrams from each text in a given time period. 

In [ ]:
from collections import Counter
import os
import pandas as pd
from gensim.models import Phrases, phrases

def getTexts(folder,searchList):
    fileToText = {}
    underscores = {}
    for root, dirs, files in os.walk(folder):
        for file in files:
                if '.txt' not in file: continue
                path = os.path.join(folder,file)
                f = open(path,'r')
                text = f.readlines()[0]
                if '_' in file: 
                        name = file.split('_')[0]
                        if name not in searchList: continue
                        if name not in underscores.keys(): 
                                underscores[name] = text
                        else: underscores[name] = underscores[name] + ' ' + text
                else: 
                        name = file.split('.')[0]
                        if name not in searchList: continue
                        fileToText[name] = text
                f.close()
        for name,text in underscores.items():
            fileToText[name] = text
        return fileToText

def collectgrams(keylist, searchword, collectlist):
        for key in keylist:
                if searchword in key.split('_'):
                        collectlist.append(key) 

def getgrams(txt):
    infile = open(txt,'r')
    lines = infile.readlines()
    infile.close()
    allgrams = []
    for line in lines: 
        line = line.split(':')
        ngrams = line[1].strip()
        if '_' in ngrams: 
                ngrams = ngrams.strip().split(' ')
                for n in ngrams:
                        allgrams.append(n)
        else: continue
    return allgrams 

In [ ]:
#change for each time period: index is period - 1
period = open('/srv/data/timeranges.txt', 'r').readlines()[4]
period = period.strip().strip('[').strip(']').replace("'", '').split(', ')
empty = []

bigramdata = getTexts('/srv/data/relevantEPBodyNOSTOP', period)
bigramtexts = list(bigramdata.values())
bigramnames = list(bigramdata.keys())
print(len(bigramnames))

In [ ]:
training = []
for t in bigramtexts:
    words = t.split(' ')
    training.append(words)

print(len(training))
bigrammodel = Phrases(training, min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
print('bigram model trained')
trigrammodel = Phrases(bigrammodel[training], min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
print('trigram model trained')

In [ ]:
#specify tobacco/drugs searchword list:
# searchwords = ['tobacco']
searchwords = ['opiate', 'opium', 'poppy', 'poppey', 'laudanum']

count = 0
outfile = open('/srv/data/joy/p5opiumngram.txt', 'w')
for text in bigramtexts:
    #for outputting to txt file, specify here
    name = bigramnames[bigramtexts.index(text)]
    testtext = text.strip().split(' ')
    grams = []

    bgcount = Counter(b for b in bigrammodel[testtext] if len(b.split("_")) > 1)
    tgcount = Counter(t for t in trigrammodel[testtext] if len(t.split("_")) > 2)
   
    for searchword in searchwords: 
        collectgrams(dict(bgcount).keys(), searchword, grams) 
        collectgrams(dict(tgcount).keys(), searchword, grams)
    
    gramstring = ' '.join(grams)
    outfile.write(name+': '+gramstring+"\n")
    count += 1
    if count%50 == 0:
        print(count, "files processed")
outfile.close()
print('processing complete')

The code below is useful for retrieving all existing n-grams, checking for thorough categorization of n-grams, as well as printing out each category list for use in social network edge colorings.

In [ ]:
p4tobacco = getgrams('/srv/data/periodFeatures/period4features/newtobaccograms.txt')
p1tobacco = getgrams('/srv/data/periodFeatures/period1features/period1tobaccongrams.txt')
p2tobacco = getgrams('/srv/data/periodFeatures/period2features/period2tobaccongrams.txt')
p3tobacco = getgrams('/srv/data/periodFeatures/period3features/period3tobaccongrams.txt')
p5tobacco = getgrams('/srv/data/periodFeatures/period5features/period5tobaccongrams.txt')

tobaccos = p1tobacco + p2tobacco + p3tobacco + p4tobacco + p5tobacco

In [ ]:
p1opium = getgrams('/srv/data/joy/p1opiumngram.txt')
p2opium = getgrams('/srv/data/joy/p2opiumngram.txt')
p3opium = getgrams('/srv/data/joy/p3opiumngram.txt')
p4opium = getgrams('/srv/data/joy/p4opiumngram.txt')
p5opium = getgrams('/srv/data/joy/p5opiumngram.txt')

drugs = p1opium + p2opium + p3opium + p4opium + p5opium


In [ ]:
#tobacco
# plant = ['call', 'clean', 'common', 'description', 'dry', 'green', 'growth', 'kind', 'leaf', 'manner', 'name', 'part', 'petum', 'piece', 'plant', 'property', 'quality', 'root', 'seed', 'small', 'sophisticate', 'sort', 'strong']
# medical = ['taste', 'loosen', 'warm', 'hot', 'set', 'almanac', 'cold', 'alymbecke', 'apply', 'bruise', 'suffumigation', 'cane', 'cure', 'draw', 'dress',  'drop', 'head', 'heat', 'help', 'humidity', 'humour', 'infusion', 'oil', 'perfume', 'phlegm', 'powder', 'prepare', 'pudding', 'purge', 'stillify', 'stir',  'vapour', 'water']
# recreation = ['face', 'arm', 'ash', 'attendance', 'box', 'break', 'breath', 'cast', 'child', 'consume', 'custom', 'cut', 'drawer',  'forget', 'fume', 'glass', 'grant', 'hard', 'have', 'invention', 'keep', 'know', 'leave', 'light', 'make', 'mark', 'paper', 'pipe', 'present', 'rake', 'savour', 'send', 'sit', 'smell', 'smoke', 'stink', 'strew', 'take', 'taker', 'touch', 'unseal', 'use', 'whiff']
# region = ['brazilian', 'dutch', 'england', 'english', 'foreign', 'indian', 'juice',  'lemnos', 'society', 'sovereign', 'sovereignty', 'spanish', 'trinidada', 'virginia']
# trade = ['alehouse', 'bag', 'barter', 'bring', 'buy', 'case', 'chest', 'commodity', 'employment', 'furnish', 'gain', 'gift', 'give', 'half', 'house', 'import', 'importation', 'licence', 'line', 'monopoly', 'ounce', 'piece', 'pocket', 'pound', 'quantity', 'retail', 'sell', 'seller', 'ship', 'shop', 'spend', 'store', 'tavern', 'trade', 'trial', 'weight']
# commodity = ['ale', 'beer', 'butter', 'chandler', 'copper', 'corn', 'cucumber', 'cup', 'drink', 'flax', 'hemlock', 'herb', 'liquorice', 'nightshade', 'other', 'rice', 'sack', 'starch', 'suet', 'sugar', 'sugarcane', 'weed', 'wine']
# malegen = ['bartholomew', 'boy', 'charles', 'chief', 'doctor', 'farmer', 'gentleman', 'his', 'main', 'maker', 'man', 'our', 'own', 'their', 'they', 'thou', 'thy', 'you', 'your']
# female = [ 'lady', 'she', 'woman'] 
# moral = ['abuse', 'against', 'bait', 'banish', 'best', 'commend', 'concern', 'counterblaste', 'delectable', 'divine', 'drone', 'entitle', 'exceed', 'excellent', 'famous', 'fast', 'few', 'force', 'good', 'great', 'hurt', 'hurtful', 'ignorant', 'immoderate', 'ingrate', 'lack', 'like', 'little', 'love', 'malady', 'many', 'mighty', 'mistake', 'more', 'much', 'must', 'noble', 'ordinary', 'ought', 'pity', 'poison', 'poor', 'power', 'praise', 'precious', 'profane', 'prove', 'reason', 'require', 'resist', 'respectless', 'right', 'save', 'scarce', 'sin', 'sleight', 'some', 'succour', 'such', 'trick', 'unwholesome', 'virtue', 'vow', 'want', 'well', 'wholesome', 'worse', 'yes']
# misc = ['far', 'one', 'two', 'about', 'after', 'afterward', 'again', 'all', 'almost', 'also', 'although', 'among', 'any', 'because', 'become', 'before', 'bind', 'both', 'bring', 'but', 'can', 'cause', 'come', 'dru', 'each', 'either', 'engender', 'especial', 'etc', 'even', 'ever', 'fac', 'fill', 'find', 'first', 'fit', 'former', 'forsooth', 'forthwith', 'from', 'here', 'heretofore', 'how', 'if', 'into', 'itself', 'let', 'likewise', 'may', 'mean', 'neither', 'nor', 'not', 'nothing', 'now', 'often', 'once', 'only', 'out', 'ready', 'regard', 'return', 'same', 'say', 'see', 'seem', 'ser', 'shall', 'still', 'sure', 'term', 'than', 'that', 'then', 'there', 'thereby', 'therefore', 'therewith', 'these', 'thing', 'think', 'this', 'those', 'though', 'through', 'till', 'tob', 'too', 'unless', 'upon', 'utter', 'very', 'what', 'whatsoever', 'when', 'where', 'whereas', 'wherein', 'wherewith', 'whether', 'which', 'while', 'who', 'why', 'will', 'within', 'yet']

In [ ]:
#opium
plant = ['bear', 'milk', 'big', 'black', 'white', 'long', 'seed', 'corn', 'crown', 'oil', 'red', 'field', 'poppy_opium', 'opium_poppy', 'flower', 'juyce', 'juice', 'leaf', 'garden', 'description', 'root', 'green', 'head', 'wild', 'thorny', 'thornye', 'kind', 'sort', 'tall', 'top', 'resemble', 'name', 'smell', 'leave', 'grow', 'horn', 'cold', 'warm', 'dry', 'sed', 'double', 'wander', 'follow']
genuse = ['custom', 'melt', 'use', 'eat', 'portion', 'quantity', 'gather', 'prepare', 'sow', 'take', 'half', 'handful', 'usage', 'serve', 'some', 'have', 'make', 'call', 'say', 'take', 'little', 'confection', 'pure', 'tablet', 'place', 'spatle', 'water', 'cast', 'consist', 'put', 'use', 'ounce', 'perfume', 'cordial', 'form', 'apply', 'anoint', 'grain', 'boil', 'cause', 'inward', 'liquid', 'join', 'camfere', 'earth', 'lay', 'mean', 'swell', 'loch', 'sod', 'equal']
medical = ['administer', 'beat', 'castoreum', 'compound', 'decoction', 'dissolve', 'dose', 'drag', 'dram', 'drink', 'dedoction', 'emplaster', 'composition', 'drowsy', 'syrup', 'give', 'tincture', 'paracelsi', 'laudanum_opiate', 'pill', 'thicken', 'diet', 'electuary', 'liquid', 'lozenge', 'medicine', 'extract', 'meconium', 'prepare', 'sleep', 'burn', 'brain']
otherplant = ['aloe', 'benzoine', 'benzoin', 'waterlilly', 'mandrake', 'lettuce', 'myrtle', 'rose', 'fig', 'frankincense', 'garlic', 'gum', 'horehound', 'wax', 'benjamin', 'hemlock', 'henbane', 'myrrh', 'violet', 'saffron', 'rosin', 'storax', 'mastic', 'oat', 'quince', 'succory', 'mandragora', 'mandrake', 'sandaraca', 'mallow', 'lettice', 'purslane', 'nightshade', 'ammoniacum', 'houseleek', 'nitre', 'bevercod', 'triphera']
region = ['english', 'french', 'cambaia', 'turkey'] 
maleppl = ['their', 'who', 'you', 'his', 'sir', 'syr', 'spirit']
female = ['she']
moral = ['against', 'bastard', 'best', 'virtue', 'clean',  'simple', 'deadly', 'scruple', 'much', 'must', 'good', 'than', 'true', 'divinity', 'prickly', 'subordinates', 'more', 'evil', 'well', 'poison', 'frothy', 'crude']
misc = ['also', 'any', 'because', 'which', 'there', 'this', 'two', 'when', 'whereof', 'like', 'each', 'even', 'first', 'one', 'foresay', 'without', 'yet', 'that', 'if', 'therein', 'thereof', 'these', 'those', 'three', 'again', 'unto', 'other', 'such', 'then', 'not', 'may', 'twice', 'two', 'according', 'but', 'shall', 'other', 'yet', 'before', 'can', 'every', 'its', 'now', 'than', 'same', 'unto', 'very', 'will', 'therefore', 'lib', 'ana', 'aun']


In [ ]:
plantgrams = []
genusegrams = []
medicalgrams = []
otherplantgrams = []
regiongrams = []
malepplgrams = []
femalegrams = []
moralgrams = []
miscgrams = []

leftover = []
categorised = []
for ngram in set(drugs):
    for word in ngram.split('_'):
        if word in plant:
            plantgrams.append(ngram)
            categorised.append(ngram)
            break
        elif word in medical:
            medicalgrams.append(ngram)
            categorised.append(ngram)
            break
        elif word in maleppl:
            malepplgrams.append(ngram)
            categorised.append(ngram)
            break
        elif word in female:
            femalegrams.append(ngram)
            categorised.append(ngram)
            break
        elif word in genuse:
            genusegrams.append(ngram)
            categorised.append(ngram)
            break
        elif word in region:
            regiongrams.append(ngram)
            categorised.append(ngram)
            break
        elif word in otherplant:
            otherplantgrams.append(ngram)
            categorised.append(ngram)
            break 
        elif word in moral:
            moralgrams.append(ngram)
            categorised.append(ngram)
            break
        elif word in misc:
            miscgrams.append(ngram)
            categorised.append(ngram)
            break
        else: 
            continue

leftover = set(drugs) - set(categorised)
print(leftover)